<h1><center>Laboratorio 8: Predicciones y Recomendaciones 🔮🪄 </center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### **Cuerpo Docente:**

- **Profesores:** Ignacio Meza, Sebastián Tinoco
- **Auxiliares:** Catherine Benavides, Consuelo Rojas
- **Ayudante:** Nicolás Ojeda, Eduardo Moya

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1:
- Nombre de alumno 2:

### **Link de repositorio de GitHub:** `http://....`

## Temas a tratar
- Series de Tiempo.
- Predicciones vía `Prophet`.
- Implementar un sistema de recomendación utilizando `surprise`.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio
- Comprender qué es una serie de tiempo y su estructura.
- Identificar tendencias, estacionalidades e irregularidades.
- Armar un modelo predictivo para la serie.
- Conocer y aplicar sistemas de recomendación.
- Entender estructura y conocer casos de estudio.


In [ ]:
# Librerias globales

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import plotly.express as px

# **Forecasting (3.0 puntos)**


## **Prophet, teoria**

Prophet es una herramienta open-source de Facebook utilizada para realizar predicciones en series de tiempo. Esta se basa en la descomposición aditiva, donde tendencias no lineales se ajustan junto a la estacionalidad.

En la ecuación de a continuación se puede ver una idea general de los elementos que construyen a un modelo aditivo como lo es Prophet.

\begin{equation}
y(t) = g(t) + s(t) + h(t) + e(t)
\end{equation}

Donde, $g(t)$ hace referencia a las tendencias, que corresponden a cambios graduales en largos periodos de tiempo. $s(t)$ corresponde a la estacionalidad, son cambios periodicos o cortos en el tiempo. $h(t)$ es el efecto que tienen las festividades sobre las predicciones, mientras que e(t) corresponde al error o ruido. Finalmente $y(t)$, es la predicción hecha por el modelo.

Prophet trabaja por defecto con Piece-Wise Lineal Model, este es un modelo de regresión lineal, en el cual se buscan distintas zonas en que la data presente patrones o tendencias lineales, de estas zonas obtiene su regresión y luego las "une" de manera de representar toda la región, como se puede ver en la ecuación siguiente.

\begin{equation}
y(x)=
    \begin{cases}
        η_1 + \beta_1(x-b_1), & b_1 < x  \leqslant b_2 \\
        η_2 + \beta_2(x-b_2), & b_2 < x  \leqslant b_3 \\
        η_3 + \beta_3(x-b_3), & b_3 < x  \leqslant b_3 \\
        ... \\
        η_k + \beta_{nb}(x-b_{nb-1}), & b_{n-1} < x  \leqslant b_{nb} \\
    \end{cases}
\end{equation}

Siendo $b_1$ el primer punto de quiebre en la serie y así hasta el punto $b_{nb}$ correspondiente al último punto de quiebre de la serie con una cantidad $nb$ de puntos.

Para mayor información de Prophet y como utilizarla, pueden ver su [documentación](https://facebook.github.io/prophet/docs/quick_start.html#python-api), donde hay pueden encontrar un pequeño tutorial de la librería.

## **Prophet, práctica**

<center>
<img src ="https://static.wikia.nocookie.net/31minutos/images/d/d1/Don_Sergio.png/revision/latest?cb=20190926222900" width = 250 />

Don Sergio quiere entrenar para participar en la maratón y como sabe que correr una maratón requiere de entrenamiento y una dieta balanceada.

Por ello, necesita empezar a comer más balanceado, pero también necesita saber que tanto le va a costar esto, de manera de poder agregarlo a su presupuesto.

Es así como ustedes tendrán que ayudar a Don Sergio a cumplir su sueño de correr la maratón de Santiago, manteniendo su economía del hogar que comparte con su espora Emerilda.

**Datos**

Para poder ayudar a Don Sergio, harán uso del dataset `fruit_vegetables.csv`que contiente las siguientes columnas:

*   `SN:` Serial Number.
*   `Commodity:` Nombre de las frutas o verduras.
* `Date:` Fecha del registro.
* `Unit:` Unidad del precio.
* `Minumum:` Precio mínimo de venta.
* `Maximum:` Precio máximo de venta.
* `Average:` Precio promedio de venta.




### **1. Serie de Tiempo**


#### **1.1 Pre-procesamiento**

Tareas de la sección:


1.   Identificar elementos necesarios para generar la serie de tiempo.
2.   Gráficar datos.
3.   Ver la existencia de tendencias, estacionalidades o ruido.
4.   Dividir datos para tener set de entrenamiento (80%) y testeo (20%). Graficar.

---





**Comida de Don Sergio**

Don Sergio quiere nutricer lo mejor posible durante su tiempo de entrenamiento, por lo que necesita agregar más frutas y verduras a su dieta. Es por ello que decidío añadir los siguientes elementos a su compra diaria.

*   Tomates
*   Lechuga
*   Manzana
*   Plantanos
*   Tofu

**Nota:** Don Sergio prefiere comprar productos locales, compra un Kg diario de cada cosa y a precio promedio.

**Respuesta:**

In [ ]:
# Leer el dataset
df=pd.read_csv('fruit_vegetables.csv')
set(df['Commodity'])

Se sabe que elige local en caso de haber varias opciones, para el caso de las manzanas hay dos tipos, por lo que solo se elegirá la que tenga precio menor en general o que tenga más datos

In [ ]:
manzanas=['Apple(Fuji)','Apple(Jholey)']
dfapple=df[df['Commodity'].isin(manzanas)].copy()
# Crear el gráfico de línea con Plotly Express
fig = px.line(dfapple, x='Date', y='Average',color='Commodity' ,title='Registros por Fecha de Reclamo')

# Mostrar el gráfico
fig.show()

Al ver el gráfico podemos ver que las manzanas del tipo jholey tienen mas datos, por lo que es este tipo el que se considerará.

In [ ]:
# Crear máscara con elementos de búsqueda y seleccionar
listacomodities=['Tofu','Tomato Small(Local)','Apple(Jholey)','Lettuce','Banana',]
dfmasc= df[df['Commodity'].isin(listacomodities)]
fig = px.line(dfmasc, x='Date', y='Average',color='Commodity' ,title='Registros por Fecha de Reclamo')

# Mostrar el gráfico
fig.show()

aquí se puede ver que todos los commodities seleccionados tienen datos por las mismas fechas

In [ ]:
# Agrupar por fecha en la columna average y sumar, obtener serie de tiempo
df_grouped_sum = dfmasc.groupby('Date')['Average'].sum().rename('Suma').copy()

In [ ]:
# Graficar serie de tiempo
fig = px.line(x=df_grouped_sum.index, y=df_grouped_sum.values  ,title='Registros por Fecha de Reclamo')

# Mostrar el gráfico
fig.show()

In [ ]:
# Realizar descomposición de la señal, utilizado seasonal_decompose
from statsmodels.tsa.seasonal import seasonal_decompose
df_descomp=seasonal_decompose(df_grouped_sum, model='additive', filt=None, period=365, two_sided=True, extrapolate_trend=0)#period es 365 porque se puede entender que es anual


In [ ]:
trend = df_descomp.trend
seasonal = df_descomp.seasonal
resid = df_descomp.resid

In [ ]:
fig_trend = px.line(x=trend.index, y=trend.values, title='Tendencia')
fig_seasonal = px.line(x=seasonal.index, y=seasonal.values, title='Estacionalidad')
fig_residual = px.line(x=resid.index, y=resid.values, title='Residuo')

# Mostrar los gráficos
fig_trend.show()
fig_seasonal.show()
fig_residual.show()

Se puede ver en los tres gráficos que existe tanto una tendencia de crecimiento, una estacionalidad en los datos y el residuo es de carácter estacionario

In [ ]:
# Realizar Hold Out de 80/20

trainsize=0.8
dfsize=len(df_grouped_sum)

dfserie_train=df_grouped_sum[:int(trainsize*dfsize)].copy()
dfserie_test=df_grouped_sum[int(trainsize*dfsize):].copy()

#### **1.2 Modelo Prophet**

A continuación se genera un modelo Prophet para la predicción. Para ello se tiene que:
1. Crear dataset, donde columna de tiempo tiene que ir en formate YYYY-MM-DD con el nombre de `ds` y columna objetivo con nombre `y`.
2. Entrenar modelo.
3. Generar predicción. Notar que el dataframe de predicción solo contiene una columna con nombre `ds`, al inicio.

In [ ]:
# librerias extras
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

**1. Generar dataframes de entrenamiento y testeo**

In [ ]:
# Respuesta
df_train=pd.DataFrame()
df_train['ds']=dfserie_train.index.copy()
df_train['y']=dfserie_train.values.copy()

df_test=pd.DataFrame()
df_test['ds']=dfserie_test.index.copy()
df_test['y']=dfserie_test.values.copy()


**2. Modelo y entrenamiento**

In [ ]:
# Respuesta, tal vez si menciono más arriba que prophet usar la sintaxis de sklearn esto lo hacen solitos nomas
model = Prophet()
model.fit(df_train)


**3. Predicción.**

¿Qué muestra el dataframe de predicción una vez realizada la predicción? ¿Qué siginifican las distintas columnas?

In [ ]:
# Respuesta
predicted=model.predict(df_test)
predicted.tail(5)

**Respuesta**

Las columnas muestran los valores de los coeficientes de la regresión que considera los componentes que según el modelo son los importantes, además de mostrar los valores para construir una banda para poder ver la variabilidad

**Gráficos de resultados**


Ahora, vemos el resultado de la predicción hecha a través de los gráficos que vienen implementados en la libreria.

Se tienen gráficos de la predicción del modelo y descomposición por componentes.

In [ ]:
# plot forecast, estático.
f, ax = plt.subplots(figsize=(15, 5))
f.set_figheight(5)
f.set_figwidth(15)

fig = model.plot(predicted, ax=ax)
plt.show()

In [ ]:
# descomposición por componentes, estático
fig = model.plot_components(predicted)

#### **1.3 Resultados y métricas**

En la presente sección se van a ver los resultados de la predicción, comparando con los resultados originales, así se obtienen métricas de error. Para ello se hacen los siguientes pasos:
1. Inspeccionar gráfica de predicción con los datos reales. Agregar tres vistas: predicción del primer 15% de los datos, 50% de los datos y 100% de la predicción.
2. Calcular métricas de error (RMSE, MAE, MAPE).

se preparan en los subconjuntos

In [ ]:
# Respuesta
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
df_test['ds'] = pd.to_datetime(df_test['ds'])
predicted['ds'] = pd.to_datetime(predicted['ds'])

testsize=len(df_test)
pred_15=predicted[:int(testsize*0.15)].copy()
df_test_15=df_test[:int(testsize*0.15)].copy()
pred_50=predicted[:int(testsize*0.5)].copy()
df_test_50=df_test[:int(testsize*0.5)].copy()


15%

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df_15 = df_test_15[['ds', 'y']].merge(pred_15[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse_15 = root_mean_squared_error(comparison_df_15['y'], comparison_df_15['yhat'])
mae_15 = mean_absolute_error(comparison_df_15['y'], comparison_df_15['yhat'])
mape_15 = np.mean(np.abs((comparison_df_15['y'] - comparison_df_15['yhat']) / comparison_df_15['y'])) * 100

print(f'RMSE: {rmse_15}')
print(f'MAE: {mae_15}')
print(f'MAPE: {mape_15}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df_15['ds'], comparison_df_15['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df_15['ds'], comparison_df_15['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

50%

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df_50 = df_test_50[['ds', 'y']].merge(pred_50[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse_50 = root_mean_squared_error(comparison_df_50['y'], comparison_df_50['yhat'])
mae_50 = mean_absolute_error(comparison_df_50['y'], comparison_df_50['yhat'])
mape_50 = np.mean(np.abs((comparison_df_50['y'] - comparison_df_50['yhat']) / comparison_df_50['y'])) * 100

print(f'RMSE: {rmse_50}')
print(f'MAE: {mae_50}')
print(f'MAPE: {mape_50}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df_50['ds'], comparison_df_50['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df_50['ds'], comparison_df_50['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

100%

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df = df_test[['ds', 'y']].merge(predicted[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse = root_mean_squared_error(comparison_df['y'], comparison_df['yhat'])
mae = mean_absolute_error(comparison_df['y'], comparison_df['yhat'])
mape = np.mean(np.abs((comparison_df['y'] - comparison_df['yhat']) / comparison_df['y'])) * 100

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df['ds'], comparison_df['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df['ds'], comparison_df['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

como se puede observar según las medidas de error, cuando solo se considera el 15% las métricas se comportan mejor, es cuando se considera el 50% que en particular para esta muestra de test las fechas contemplan los primeros meses de la pandemia, el error sube casi al doble debido a la discrepancia de lo predicho con los real, una vez se considera todo el conjunto de test, baja un poco el error, porque los precios son más parecidos a los presentes en los años anteriores y por ende la predicción es mejor.

#### **1.4 Festividades y fechas importantes**

Como a todos, a Don Sergio también le afectan las fechas importantes. Para ver como afectan estas fechas al presupuesto de Don Sergio tiene que realizar las siguientes tareas:

1. Crear dataframe con fechas importantes y agregar las al modelo con el argumento `holiday`.
2. Predecir las el dataframe de testeo
3. Comparar resultados. Hacer una tabla con resultados RMSE, MAE, MAPE anteriores y al añadir las festividades.


**1. Festividades y agregar a modelo**

Suponiendo que Don Sergio vive en Chile se consideran fechas importantes, las fiestas patrias, año nuevo, semana santa, navidad. No se consideran todas los feriados

In [ ]:

fechas_importantes_lista = [
    "2013-01-01",
    "2014-01-01",
    "2015-01-01",
    "2016-01-01",
    "2017-01-01",
    "2018-01-01",
    "2019-01-01",
    "2020-01-01",

    "2013-09-18",
    "2014-09-18",
    "2015-09-18",
    "2016-09-18",
    "2017-09-18",
    "2018-09-18",
    "2019-09-18",
    "2020-09-18",

    "2013-09-19",
    "2014-09-19",
    "2015-09-19",
    "2016-09-19",
    "2017-09-19",
    "2018-09-19",
    "2019-09-19",
    "2020-09-19",

    # Semana Santa (Viernes Santo y Domingo de Resurrección)
    "2013-03-29", "2013-03-30", "2013-03-31",
    "2014-04-18", "2014-04-19","2014-04-20",
    "2015-04-03", "2015-04-04","2015-04-05",
    "2016-03-25", "2016-03-26","2016-03-27",
    "2017-04-14", "2017-04-15","2017-04-16",
    "2018-03-30", "2018-03-31","2018-04-01",
    "2019-04-19", "2019-04-20","2019-04-21",
    "2020-04-10", "2020-04-11","2020-04-12",
    "2021-04-02", "2021-04-03","2021-04-04",
    # Navidad
    "2013-12-25",
    "2014-12-25",
    "2015-12-25",
    "2016-12-25",
    "2017-12-25",
    "2018-12-25",
    "2019-12-25",
    "2020-12-25"]

fechas_importantes = pd.DataFrame({
    'ds': pd.to_datetime(fechas_importantes_lista),
    'holiday': 'feriado'
})

In [ ]:
# Respuesta
df_holi=pd.DataFrame()
df_holi['ds']=df_grouped_sum.index.copy()
df_holi['y']=df_grouped_sum.values.copy()

se realiza el mismo holdout 80/20

In [ ]:
# Realizar Hold Out de 80/20

trainsize=0.8
dfsize=len(df_holi)

df_holi_train=df_holi[:int(trainsize*dfsize)].copy()
df_holi_test=df_holi[int(trainsize*dfsize):].copy()

**2. Predecir**

In [ ]:
# Respuesta
model2 = Prophet(holidays=fechas_importantes)
model2.fit(df_holi_train)

predicted_holi=model2.predict(df_holi_test)


**3. Resultados**

In [ ]:
fig = model2.plot_components(predicted_holi)

In [ ]:
# Respuesta
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
df_holi_test['ds'] = pd.to_datetime(df_holi_test['ds'])
predicted_holi['ds'] = pd.to_datetime(predicted_holi['ds'])

testsize_holi=len(df_holi_test)
pred_holi_15=predicted_holi[:int(testsize_holi*0.15)].copy()
df_holi_test_15=df_holi_test[:int(testsize_holi*0.15)].copy()
pred_holi_50=predicted_holi[:int(testsize_holi*0.5)].copy()
df_holi_test_50=df_holi_test[:int(testsize_holi*0.5)].copy()

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df_holi_15 = df_holi_test_15[['ds', 'y']].merge(pred_holi_15[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse_holi_15 = root_mean_squared_error(comparison_df_holi_15['y'], comparison_df_holi_15['yhat'])
mae_holi_15 = mean_absolute_error(comparison_df_holi_15['y'], comparison_df_holi_15['yhat'])
mape_holi_15 = np.mean(np.abs((comparison_df_holi_15['y'] - comparison_df_holi_15['yhat']) / comparison_df_holi_15['y'])) * 100

print(f'RMSE: {rmse_holi_15}')
print(f'MAE: {mae_holi_15}')
print(f'MAPE: {mape_holi_15}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df_holi_15['ds'], comparison_df_holi_15['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df_holi_15['ds'], comparison_df_holi_15['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df_holi_50 = df_holi_test_50[['ds', 'y']].merge(pred_holi_50[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse_holi_50 = root_mean_squared_error(comparison_df_holi_50['y'], comparison_df_holi_50['yhat'])
mae_holi_50 = mean_absolute_error(comparison_df_holi_50['y'], comparison_df_holi_50['yhat'])
mape_holi_50 = np.mean(np.abs((comparison_df_holi_50['y'] - comparison_df_holi_50['yhat']) / comparison_df_holi_50['y'])) * 100

print(f'RMSE: {rmse_holi_50}')
print(f'MAE: {mae_holi_50}')
print(f'MAPE: {mape_holi_50}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df_holi_50['ds'], comparison_df_holi_50['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df_holi_50['ds'], comparison_df_holi_50['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df_holi = df_holi_test[['ds', 'y']].merge(predicted_holi[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse_holi = root_mean_squared_error(comparison_df_holi['y'], comparison_df_holi['yhat'])
mae_holi = mean_absolute_error(comparison_df_holi['y'], comparison_df_holi['yhat'])
mape_holi = np.mean(np.abs((comparison_df_holi['y'] - comparison_df_holi['yhat']) / comparison_df_holi['y'])) * 100

print(f'RMSE: {rmse_holi}')
print(f'MAE: {mae_holi}')
print(f'MAPE: {mape_holi}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df_holi['ds'], comparison_df_holi['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df_holi['ds'], comparison_df_holi['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

In [ ]:
metrics_values = {
    'RMSE': {
        'Original': {
            '100%': rmse,
            '50%': rmse_50,
            '15%': rmse_15
        },
        'Holiday': {
            '100%': rmse_holi,
            '50%': rmse_holi_50,
            '15%': rmse_holi_15
        }
    },
    'MAE': {
        'Original': {
            '100%': mae,
            '50%': mae_50,
            '15%': mae_15
        },
        'Holiday': {
            '100%': mae_holi,
            '50%': mae_holi_50,
            '15%': mae_holi_15
        }
    },
    'MAPE': {
        'Original': {
            '100%': mape,
            '50%': mape_50,
            '15%': mape_15
        },
        'Holiday': {
            '100%': mape_holi,
            '50%': mape_holi_50,
            '15%': mape_holi_15
        }
    }
}

# Crear DataFrames para cada métrica
rmse_df = pd.DataFrame(metrics_values['RMSE'])
mae_df = pd.DataFrame(metrics_values['MAE'])
mape_df = pd.DataFrame(metrics_values['MAPE'])

# Mostrar las matrices de métricas
print("RMSE:")
print(rmse_df)
print("\nMAE:")
print(mae_df)
print("\nMAPE:")
print(mape_df)

¿Son más precisas las predicciones? ¿Qué otro festividad podría afectar a los precios de las frutas y verduras?

**Respuesta**
Las predicciones son ligeramente más precisas, aunque solo es una diferencia decimal en comparación al modelo sin holidays, es cerca de un 0.2 en RMSE para todos los casos, 0.15 de la misma forma para MAE y para MAPE que es porcentual es cerca de 0.07


Con respecto a las fechas o festividades, tal vez podrían afectar fechas conflictivas, como el día del joven combatiente, o el 11 de septiembre estas podrían afectar los precios indirectamente, al alza.



#### **1.5 Regresores adicionales**


Don Sergio sabe que la fruta y la verdura tiene temporadas y que por eso hay fechas en que hay ciertas cosas más caras que otras. Además, sabe que este tipo de alimentos sube de precio en invierno, por lo que necesita que tenga en cuenta estas fechas.


---
 Para lograr lo anterior tendrán que:
 1. Crear regresor con 1 y 0's según corresponda, para las fechas de invierno. Tomar invierno desde el 21/junio hasta el 21/septiembre de todos los años.
 2. Agregar columna con el regresor tanto al dataframe original como al de testeo
 3. Entrenar y predecir. Gráficar resultados.

como el titulo dice adicionales, se supondra que se trabaja sobre el modelo con holidays

In [ ]:
# Crear una lista de fechas de invierno para todos los años
fechas_invierno = []
for year in range(2013, 2022):
    fechas_invierno.extend(pd.date_range(start=f'21-06-{year}', end=f'21-09-{year}'))

# Convertir la lista de fechas a un DataFrame
regresor_invierno_df = pd.DataFrame({
    'ds': fechas_invierno,
    'invierno': 1  # Asignar 1 para las fechas de invierno
})

In [ ]:
df_holi2=df_holi.copy()
df_holi2['ds'] = pd.to_datetime(df_holi2['ds'])
df_holi2['mes_dia'] = df_holi2['ds'].dt.strftime('%m-%d')
fechas_invierno_mes_dia = regresor_invierno_df['ds'].dt.strftime('%m-%d')
df_holi2['invierno'] = df_holi2['mes_dia'].isin(fechas_invierno_mes_dia).astype(int)
del df_holi2['mes_dia']

df_holi2_train=df_holi2[:int(trainsize*dfsize)].copy()
df_holi2_test=df_holi2[int(trainsize*dfsize):].copy()

In [ ]:
model3 = Prophet(holidays=fechas_importantes)
model3.add_regressor('invierno')
model3.fit(df_holi2_train)

predicted_holi2=model3.predict(df_holi2_test)

fig = model3.plot_components(predicted_holi2)

In [ ]:
# Alinear las predicciones con los datos reales usando merge
comparison_df_holi2 = df_holi2_test[['ds', 'y']].merge(predicted_holi2[['ds', 'yhat']], on='ds', how='left')

# Calcular las métricas de error
rmse_holi2 = root_mean_squared_error(comparison_df_holi2['y'], comparison_df_holi2['yhat'])
mae_holi2 = mean_absolute_error(comparison_df_holi2['y'], comparison_df_holi2['yhat'])
mape_holi2 = np.mean(np.abs((comparison_df_holi2['y'] - comparison_df_holi2['yhat']) / comparison_df_holi2['y'])) * 100

print(f'RMSE: {rmse_holi2}')
print(f'MAE: {mae_holi2}')
print(f'MAPE: {mape_holi2}%')

# Graficar los resultados
fig, ax = plt.subplots(figsize=(15, 5))

# Graficar los valores reales
ax.plot(comparison_df_holi2['ds'], comparison_df_holi2['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
ax.plot(comparison_df_holi2['ds'], comparison_df_holi2['yhat'], label='Pronóstico', color='red')

# Configurar etiquetas y título
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.set_title('Comparación entre Datos Reales y Pronóstico')
ax.legend()

# Mostrar el gráfico
plt.show()

Al agregar el regresor de la temporada de invierno, se puede observar una leve mejora de las métricas de error, de 0.31 para RMSE, 0.36 para MAE y 0.1 para MAPE, con respecto al  100% de las predicciones del modelo que considera los holiday

Finalmente mencionar que sería interesante poder agregar como fechas a considerar con un regresor aparte, cuando hubo cuarentena total en el país, ya que las principales caídas de los precios coinciden con estas medidas el primer año de pandemia, similar con esto poder considerar los días que se aprobaron los retiros de pensiones en consecuencia de la pandemia, tal vez muestren las subidas repentinas en el conjunto de test. 

# **Sistemas de Recomendación (3.0 puntos)**

<center>
<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Shopping_online_with_bank_card.jpg/1024px-Shopping_online_with_bank_card.jpg" / width = 350>

Como no todo en la vida es correr y nutrirse, don Sergio desarrollo el hábito de comprar online y pasa horas y horas en Amazon viendo distintos objetos que le podrían servir, pero no sabe qué comprar, por lo que usted tendrá que ayudarlo a buscar los elementos más comprados por sus amigos para que él también pueda probarlos.

---

Las tareas a realizar son:

1. Limpiar y explorar el dataset.
2. Recomendación por Ranking (baseline).
3. Recomendación basada en filtros colaborativos por usuarios.
4. Recomendaciones basadas en filtros colaborativos por items.



In [ ]:
# se instala surprise library
!pip install surprise

In [ ]:
# Librerias extras
from sklearn.preprocessing import LabelEncoder

from surprise import accuracy
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVD

import pandas as pd
from copy import deepcopy

Los datos presentan las siguientes columnas:
* `iderId`: ID único de cada usuario
* `productId:` ID único de cada producto
* `Rating:` Rating que un usuario le dio a un producto
* `timestamp:` Fecha del rating

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_parquet('ratings_Electronics.parquet')
df.head()

In [ ]:
null_count = df.isnull().sum().sum()
print('Number of null values:', null_count)
print('Size:', df.shape)

In [ ]:
df.dtypes

### **2.1.2 EDA (0.5 puntos)**

Habiendo limpiado los datos, generemos un pequeño análisis exploratorio:

Reportar la cantidad de usuarios y productos únicos:

In [ ]:
num_users = df['user_id'].nunique()
num_products = df['prod_id'].nunique()

print(f"Cantidad de usuarios únicos: {num_users}")
print(f"Cantidad de productos únicos: {num_products}")

Verificar que no hayan valores nulos

In [ ]:
null_counts = df.isnull().sum()

print("Cantidad de valores nulos por columna:")
print(null_counts)

Graficar distribución de los ratings

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['rating'], bins=[1, 2, 3, 4, 5, 6], edgecolor='k', alpha=0.7, align='left', rwidth=0.8)
plt.title('Distribución de Ratings')
plt.xlabel('Rating')
plt.ylabel('Frecuencia')
plt.xticks([1, 2, 3, 4, 5])
plt.grid(True)
plt.show()

Reportar las siguientes listas:

- Top 5 de usuarios con mayores interacciones
- Top 5 de productos con mayores inteacciones

In [ ]:
top_users = df.groupby('user_id')['rating'].count().sort_values(ascending=False).head(5)
print("Top 5 de usuarios con mayores interacciones:")
print(top_users)

top_products = df.groupby('prod_id')['rating'].count().sort_values(ascending=False).head(5)
print("Top 5 de productos con mayores interacciones:")
print(top_products)

A partir de sus respuestas, escriba al menos 3 conclusiones.

Escriba sus observaciones aquí:

1. Es importante observar que no existen valores nulos. Estos podrían ser un problema al momento de entrenar modelos.
2. El dataset tiene 164 mil filas, y sólo hay 8 mil usuarios y cerca de 7 mil productos. Es decir, cada usuario debe tener muchas interacciones con distintos productos, como se ve en el top 5 hecho.
3. Hay una gran tendencia en que los productos estén calificados con 5 estrellas, pues la distribución de rating está concentrado hacia calificaciones más altas. Esto podría ser un problema si es que se busca recomendar productos con cierto puntaje, pues la gran mayoria tiene el puntaje máximo.

## **2.2 Holdout**

Con los datos procesados, separe los datos de entrenamiento en train y test asignando 30% de los datos para el conjunto de test. Para replicabilidad de sus respuestas, fije la semilla en `42`.

In [ ]:
reader = Reader(rating_scale=(0,5))

df_collaborative = df[['user_id', 'prod_id', 'rating']].copy() # ordenamos dataset en user, producto, rating (paso clave para trabajar con Surprise)
dataset = Dataset.load_from_df(df_collaborative, reader) # generamos Dataset de Surprise

In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(dataset, test_size=0.3, random_state=42)

num_train_ratings = sum(1 for _ in trainset.all_ratings())
num_test_ratings = len(testset)
print(f"Tamaño del conjunto de entrenamiento: {num_train_ratings}")
print(f"Tamaño del conjunto de prueba: {num_test_ratings}")

## **2.3 Baseline**

El objetivo de esta sección es generar un *baseline* en las recomendaciones a partir de una aproximación *dummy* para resolver el problema de recomendación. En particular, se le pide lo siguiente:
- Entrenar modelo baseline `NormalPredictor` de `surprise` con los datos de entrenamiento
- Generar predicciones usando el modelo entrenado
- Evaluar predicciones usando el error cuadrático medio. Entregue una interpretación de su resultado

In [ ]:
from surprise.prediction_algorithms.random_pred import NormalPredictor

model = NormalPredictor()
model.fit(trainset)
predictions = model.test(testset)
pd.DataFrame(predictions)

In [ ]:
# Evaluar con MAE
from surprise.accuracy import mae
mae(predictions)

Un MAE de 0.9919 significa que, en promedio, las predicciones del modelo están a 0.9919 unidades de distancia de los valores reales. Dado que los ratings en el dataset van de 1 a 5, un error promedio de 0.9919 es relativamente bajo, ya que la diferencia máxima posible es de 4 (si alguien predice 1 y el valor real es 5 o viceversa). En términos prácticos, esto significa que si, por ejemplo, un usuario da un rating de 4 a un producto, el modelo, en promedio, predice algo entre aproximadamente 3 y 5 (4 ± 0.9919). Un MAE cercano a 1 indica que el modelo está bastante cerca de los valores reales, pero todavía hay margen para mejorar.


## **Filtros Colaborativos Basados en Memoria**

A modo de mejorar esta sección, se le pide que implementar filtros colaborativos basados en memoria por medio de algoritmos basados en KNN. En específico, se le pide implementar:
- Filtro Colaborativo KNN enfocado en el Usuario
- Filtro Colaborativo KNN enfocado en el Producto
- Use `KNNBasic` y similitud coseno para ambos modelos

Luego **para cada uno de los modelos**, responda:

- Obtenga el error cuadrático medio de sus recomendaciones y comente. ¿Qué enfoque tienen un mejor rendimiento? ¿Se alcanza un mejor rendimiento con respecto al Baseline?
- Obtenga el rating que le asignaria el usuario `3` al producto `3906`. ¿Son diferentes sus resultados? ¿Porqué?
- Obtenga las 5 mejores recomendaciones para el usuario `2170` usando ambos enfoques y compare sus resultados.

*Hint: Le podría servir apoyarse de esta [documentación](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration).*

In [ ]:
def get_user_top_k(predictions, user, k = 10):

  # predecir los rating filtrados por usuario user
  predicted_ratings = [pred for pred in predictions if pred.uid == user]

  # Ordenar las predicciones
  sorted_predicted_ratings = sorted(predicted_ratings, key=lambda x: x.est, reverse=True)

  return sorted_predicted_ratings[:k]

In [ ]:
from surprise.accuracy import mse

# entrenar algoritmo en base a usuario

sim_options_user = {'name': 'cosine',
               'user_based': True}

# algoritmo base a usuario.
user_model =  KNNBasic(sim_options=sim_options_user)

# entrenar
user_model.fit(trainset)

#predecir en dataframe de testeo

user_predictions =  user_model.test(testset) # retornar un dataframe con el rating estimado para cada (user_id, item_id)

# obtener mse
mse_user = accuracy.mse(user_predictions)

In [ ]:
# ejemplo
example = [(3, 3906, None)]
print(user_model.test(example))
get_user_top_k(user_predictions, user = 2170, k = 5)

In [ ]:
# entrenar algoritmo base a items, obtener MSE

sim_options_item = {'name': 'cosine', 'user_based': False}

# algoritmo base a items.

item_model = KNNBasic(sim_options=sim_options_item)

# entrenar
item_model.fit(trainset)

#predecir en dataframe de testeo

item_predictions = item_model.test(testset)

# obtener mse
mse_item = accuracy.mse(item_predictions)

In [ ]:
example = [(3, 3906, None)]
print(item_model.test(example))

get_user_top_k(item_predictions, user = 2170, k = 5)

*  En este caso, el MSE más bajo indica un mejor rendimiento, ya que representa una menor discrepancia entre las predicciones del modelo y los valores reales. Dado que el MSE del filtro colaborativo KNN basado en usuario (1.2122) es menor que el MSE del filtro colaborativo KNN basado en ítem (1.2328), podemos concluir que el enfoque basado en usuario tiene un mejor rendimiento en este escenario. Luego, como el MSE del Baseline es menor que los MSE de los modelos KNN, entonces el Baseline tiene un mejor rendimiento.

*  Los resultados son diferentes para el mismo usuario y producto entre los enfoques basados en usuario y en ítem. Esto se debe a las diferencias en cómo se calculan las similitudes entre usuarios e ítems y cómo se utilizan esas similitudes para generar predicciones.

* Algunos productos recomendados son los mismos en ambos enfoques, como los productos con ID 1326, 2679 y 4650. Sin embargo, el orden de las recomendaciones puede variar. También hay diferencias en los productos recomendados entre los enfoques. Por ejemplo, en el enfoque usuario, el producto con ID 4363 es recomendado, mientras que en el enfoque ítem, el producto con ID 3595 es recomendado. Los ratings estimados para los mismos productos pueden variar entre los enfoques. Por ejemplo, para el producto con ID 1326, el rating estimado es 4.7998 en el enfoque usuario y 5 en el enfoque ítem. Aunque los productos recomendados pueden ser los mismos, los ratings estimados pueden ser diferentes debido a las diferencias en cómo se calculan las predicciones en cada enfoque.

## **Filtros Colaborativos Basados en Modelo**

Finalmente, se le pide implementar el modelo `SVD` para resolver el problema. Al igual que en las secciones anteriores, entrene este modelo y luego evalúe sus predicciones en el conjunto de test. Luego, responda:

- ¿En qué se diferencia este tipo de modelos de los modelos basados en memoria?
- ¿Qué significa el parámetro `num_factors`? ¿De qué manera podría impactar en el ajuste del modelo?
- ¿Qué indican los parámetros $\mu$, $b_u$, $b_i$? ¿Qué cuidado se debe tener al utilizarlos?
- Obtenga el error cuadrático medio de sus recomendaciones y comente. ¿Se alcanza un mejor rendimiento con respecto al Baseline?
- Obtenga el rating que le asignaria el usuario `3` al producto `3906`. ¿Son diferentes sus resultados? ¿Porqué?
- Obtenga las 5 mejores recomendaciones para el usuario `2170` usando ambos enfoques y compare sus resultados.

In [ ]:
# Respuesta
svd_model = SVD()

svd_model.fit(trainset)

# predicciones, retornar un dataframe con el rating estimado para cada (user_id, item_id)

svd_predictions = svd_model.test(testset)

In [ ]:
# calcular mae
from surprise.accuracy import mae
mae(svd_predictions)

In [ ]:
example = [(3, 3906, None)]
print(svd_model.test(example))
get_user_top_k(svd_predictions, user = 2170, k = 5)

* Los modelos basados en memoria utilizan medidas de similitud para calcular la similitud entre usuarios o ítems y generar recomendaciones. Además, estos modelos pueden enfrentar problemas de escalabilidad y esparcidad en conjuntos de datos grandes y dispersos, ya que calculan la similitud entre cada par de usuarios o ítems. Por otra parte, los modelos SVD utilizan técnicas de factorización de matrices para aprender patrones latentes en los datos y generar recomendaciones basadas en esos patrones. Las recomendaciones se generan utilizando los factores latentes aprendidos, que pueden ser más abstractos y difíciles de interpretar directamente. Además, Los modelos SVD son más escalables y pueden manejar de manera más efectiva la esparcidad de los datos al factorizar la matriz de interacciones.

* El parámetro `num_factors` en el modelo SVD representa el número de factores latentes que el modelo utilizará para representar tanto a los usuarios como a los ítems en el espacio de características latentes. Cada factor latente captura una dimensión abstracta o característica oculta que influye en la interacción entre usuarios e ítems. El impacto del parámetro `num_factors` en el ajuste del modelo es significativo y está relacionado con la capacidad del modelo para capturar la complejidad y las características latentes presentes en los datos de interacción usuario-ítem. Un valor más alto permitirá al modelo capturar más complejidad en los datos, ya que habrá más factores latentes disponibles para modelar las interacciones entre usuarios e ítems. Sin embargo, un valor excesivamente alto puede llevar a un sobreajuste del modelo, especialmente en conjuntos de datos pequeños, donde el modelo puede aprender ruido en lugar de patrones significativos. Un valor demasiado bajo de `num_factors` puede llevar a un modelo simplificado que no puede capturar completamente la estructura latente en los datos, lo que resulta en un rendimiento deficiente en el conjunto de prueba. El número de factores latentes afecta directamente la complejidad computacional del modelo. Cuantos más factores latentes haya, más operaciones computacionales serán necesarias para entrenar y hacer predicciones con el modelo.

* $\mu$ representa la media global de todos los ratings en el conjunto de datos. Es decir, es el promedio de todos los ratings dados por todos los usuarios a todos los ítems. Al utilizar $\mu$, es importante considerar si se desean centrar los datos o no. Centrar los datos puede ser útil para capturar las desviaciones individuales de la media global, pero también puede eliminar información importante sobre la distribución de los ratings.

* $b_u$ representa el sesgo de un usuario específico en sus ratings. Es decir, es la desviación promedio de los ratings dados por un usuario en comparación con la media global $\mu$. Al utilizar $b_u$, es importante considerar el efecto del sesgo de usuario en el modelo. Un valor de $b_u$ positivo indica que el usuario tiende a dar ratings más altos que la media global, mientras que un valor negativo indica lo contrario.

* $b_i$ representa el sesgo de un ítem específico en sus ratings. Es decir, es la desviación promedio de los ratings dados a un ítem en comparación con la media global $\mu$. Al utilizar $b_i$, es importante considerar el efecto del sesgo de ítem en el modelo. Un valor de $b_i$ positivo indica que el ítem tiende a recibir ratings más altos que la media global, mientras que un valor negativo indica lo contrario.

* El modelo SVD tiende a cometer errores de aproximadamente 0.6934 unidades al predecir los ratings de los usuarios en el conjunto de prueba en este caso, po lo que alcanza un mejor rendiminento que el baseline.

* Sí, los resultados son diferentes entre el Baseline y el modelo SVD. La predicción para el rating que el usuario 3 asignaría al producto 3906 es aproximadamente 3.0372 según el modelo SVD. La diferencia en los resultados puede deberse a las diferentes aproximaciones y técnicas utilizadas por cada modelo. Por ejemplo, el modelo SVD utiliza una descomposición de valor singular para aprender los patrones latentes en los datos de interacción usuario-ítem.

* Aunque hay cierta superposición en los productos recomendados entre los enfoques, las diferencias en los ratings estimados y en el orden de las recomendaciones muestran que los resultados pueden variar según el enfoque utilizado. Esto destaca la importancia de evaluar y comparar múltiples enfoques en sistemas de recomendación para encontrar el más adecuado para un conjunto de datos y un contexto específico.

###**Comentarios Finales**

* ¿Qué sistema de recomendación presenta mejores métricas? ¿A qué se podría deber esto?

El modelo SVD presenta el MAE más bajo de todos los sistemas de recomendación evaluados. Esto indica que, en promedio, las predicciones del modelo SVD están más cerca de los valores reales en el conjunto de prueba en comparación con los otros sistemas de recomendación. El modelo SVD utiliza una técnica avanzada de factorización de matrices para capturar patrones latentes en los datos de interacción usuario-ítem, lo que puede permitir una mejor comprensión de las preferencias de los usuarios y las características de los ítems. Además, este modelo puede generalizar mejor a partir de los datos de entrenamiento y evitar el sobreajuste en comparación con los filtros colaborativos basados en memoria, lo que puede conducir a un mejor rendimiento en datos no vistos.

* ¿Es posible mejorar sus resultados? Si es así, proponga al menos 2 mejoras.

Sí, es posible mejorar los resultados del modelo SVD y de otros sistemas de recomendación. El modelo SVD y otros sistemas de recomendación a menudo tienen hiperparámetros que pueden ajustarse para mejorar su rendimiento.
Por ejemplo, en el caso del modelo SVD, se puede ajustar `num_factors` para encontrar el valor óptimo que equilibre la complejidad del modelo y su capacidad para capturar patrones en los datos. Además, otros hiperparámetros, como la tasa de aprendizaje y el número de iteraciones en el proceso de optimización, pueden ajustarse para mejorar el rendimiento del modelo.
Se podría experimentar con diferentes valores de hiperparámetros y utilizar técnicas de validación cruzada para evaluar el impacto que ocasionan en las métricas de evaluación, y así encontrar la configuración óptima del modelo.